In [2]:
# %pip install -q -U git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets accelerate wandb

In [3]:
# %pip install -q wandb

In [4]:
import wandb

In [5]:
# %cd dataset
# !unrar e /content/drive/MyDrive/Dataset/Datasets.rar
# %cd ..

In [6]:
import datasets
import pandas as pd
from pathlib import Path
from PIL import Image
from transformers import AutoTokenizer
from transformers import Blip2Processor, Blip2VisionModel, Blip2QFormerModel, Blip2QFormerConfig, Blip2ForConditionalGeneration
from transformers import AutoProcessor, Blip2ForConditionalGeneration

import torch

c:\Users\user\miniconda3\envs\imageCaptioning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", #TODO put back in "load_in_8bit" for model
    device_map={"": 0},
    torch_dtype=torch.float16
)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from peft import LoraConfig, get_peft_model,LoftQConfig

loftQConfig = LoftQConfig(loftq_bits=4)
# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"],
    init_lora_weights="loftq",
    loftq_config=loftQConfig
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

NameError: name 'model' is not defined

In [ ]:
images_path = list(Path("/content/dataset").glob("*.jpg"))
labels = pd.read_csv("/content/dataset/preprocess.csv")
images = [Image.open(str(images_path[0].parent / path)) for path in labels['image']]
dataset = datasets.Dataset.from_dict({"image": images, "text": labels['caption']})
dataset = dataset.train_test_split(test_size=0.1, seed=42)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["text"]
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

In [ ]:
train_dataset = ImageCaptioningDataset(dataset['train'], processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

test_dataset = ImageCaptioningDataset(dataset['test'], processor)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

In [ ]:
import torch
from tqdm import tqdm
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()

for epoch in range(10):
    training_loss = 0
    print("Epoch:", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader)):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)
        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss
        training_loss += loss.item()
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
    print(training_loss / len(train_dataloader))

Epoch: 0


461it [06:38,  1.16it/s]


KeyboardInterrupt: 

In [ ]:
im = dataset['test'].select([290])['image']

In [ ]:
pixel_values = processor(im, return_tensors="pt").to(device).pixel_values
model.eval()

outputs = model.generate(pixel_values=pixel_values)

In [ ]:
generated_caption = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print(generated_caption)

 สุนัขสีน�
